---

## 1. Filtrando e tratando os dados

---

In [ ]:
import pandas as pd

df = pd.read_csv('movies.csv')
print(f"Arquivo 'movies.csv' carregado com {len(df)} linhas.")

Arquivo 'movies.csv' carregado com 722317 linhas.


In [24]:
# Processando a coluna 'release_date'
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month
df['release_day_of_week'] = df['release_date'].dt.dayofweek

In [25]:
# Processando a coluna 'genres'
df['genres'] = df['genres'].fillna('')
genres_dummies = df['genres'].str.get_dummies(sep='-').add_prefix('genre_')
df = pd.concat([df, genres_dummies], axis=1)
df = df.drop('genres', axis=1)

In [26]:
# Limpeza de dados numéricos ('budget' e 'revenue')
linhas_antes_da_limpeza = len(df)
df = df[(df['budget'] > 0) & (df['revenue'] > 0)]

print(f"Linhas antes da limpeza de orçamento: {linhas_antes_da_limpeza}")
print(f"Linhas após a limpeza (dataset final): {len(df)}")
print(f"Total de {linhas_antes_da_limpeza - len(df)} linhas foram removidas.")
print("\nVisualização das primeiras 5 linhas do DataFrame final e limpo:")
print(df[['id', 'budget', 'revenue', 'release_year', 'genre_Action']].head())

Linhas antes da limpeza de orçamento: 722317
Linhas após a limpeza (dataset final): 11277
Total de 711040 linhas foram removidas.

Visualização das primeiras 5 linhas do DataFrame final e limpo:
       id       budget       revenue  release_year  genre_Action
0  615656  129000000.0  3.520565e+08        2023.0             1
1  758323   18000000.0  6.567582e+07        2023.0             0
2  533535  200000000.0  1.326387e+09        2024.0             1
3  667538  200000000.0  4.070455e+08        2023.0             1
4  693134  190000000.0  6.838137e+08        2024.0             0


In [27]:
df.to_csv('filmes_tratado.csv', index=False, encoding='utf-8-sig')

---

## 2. Adicionando novas Features

---

In [28]:
import pandas as pd
import numpy as np
import ast
from collections import Counter

df = pd.read_csv('filmes_tratado.csv')

### Tratamento

In [29]:
# Substituir orçamentos implausivelmente baixos por NaN e Preencher NaNs de orçamento com a mediana
df['budget'] = df['budget'].replace(0, np.nan)
df['budget'] = df['budget'].fillna(df['budget'].median())

def safe_literal_eval(val):
    try:
        if isinstance(val, str):
            return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return []
    return []

print("Processando colunas complexas (keywords, production_companies)...")
df['keywords_list'] = df['keywords'].apply(safe_literal_eval)
df['companies_list'] = df['production_companies'].apply(safe_literal_eval)

Processando colunas complexas (keywords, production_companies)...


### Criando Features

In [30]:
# a. Features baseadas em contagem
df['num_keywords'] = df['keywords_list'].apply(len)
df['num_companies'] = df['companies_list'].apply(len)
df['has_tagline'] = df['tagline'].notna().astype(int)
df['overview_length'] = df['overview'].astype(str).str.len()

# Features para Identificar keywords e estúdios mais influentes
all_keywords = [item['name'] for sublist in df['keywords_list'] for item in sublist]
top_keywords = [item[0] for item in Counter(all_keywords).most_common(30)] # Top 30 keywords

all_companies = [item['name'] for sublist in df['companies_list'] for item in sublist]
top_companies = [item[0] for item in Counter(all_companies).most_common(20)] # Top 20 estúdios

print("Criando features para top keywords e estúdios...")
for keyword in top_keywords:
    df[f'keyword_{keyword.replace(" ", "_")}'] = df['keywords'].str.contains(f"'{keyword}'", regex=False, na=False)

for company in top_companies:
    df[f'company_{company.replace(" ", "_")}'] = df['production_companies'].str.contains(f"'{company}'", regex=False, na=False)

Criando features para top keywords e estúdios...


### Seleção e Salvamento

In [31]:
features_to_keep = [
    'popularity', 'budget', 'runtime', 'vote_count', 'vote_average', # Incluindo o alvo
    'release_year', 'release_month', 'release_day_of_week'
]

genre_cols = [col for col in df.columns if col.startswith('genre_')]
features_to_keep.extend(genre_cols)

new_feature_cols = [
    'num_keywords', 'num_companies', 'has_tagline', 'overview_length'
]
features_to_keep.extend(new_feature_cols)

top_keyword_cols = [col for col in df.columns if col.startswith('keyword_')]
features_to_keep.extend(top_keyword_cols)

top_company_cols = [col for col in df.columns if col.startswith('company_')]
features_to_keep.extend(top_company_cols)

df_tratado = df[list(set(features_to_keep))].copy() 
df_tratado.replace([np.inf, -np.inf], np.nan, inplace=True)
df_tratado.fillna(0, inplace=True)

In [32]:
output_filename = 'filmes_tratado_novas_features.csv'
df_tratado.to_csv(output_filename, index=False)

print(f"\nTratamento concluído!")
print(f"O arquivo '{output_filename}' foi criado com sucesso.")


Tratamento concluído!
O arquivo 'filmes_tratado_novas_features.csv' foi criado com sucesso.
